In [3]:
import nba_api.stats.endpoints as nbaapi
import rdshelp
import pandas as pd

# ✅ RDS CONNECTION NOW WORKING! (Fixed security group)
print("🔍 Testing RDS connection after security group fix...")

conn = rdshelp.connect_to_rds('thebigone', 'ajwin', 'CharlesBark!23', 'nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com')


🔍 Testing RDS connection after security group fix...
Connected to RDS PostgreSQL database


In [ ]:
gamefinder = nbaapi.leaguegamefinder.LeagueGameFinder(
                            league_id_nullable='00',
                            season_nullable='2024-2025')

In [16]:
gamefinder.get_data_frames()[0].to_csv('test.csv')

In [ ]:
# Query all public tables in the connected RDS database

query = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public'
ORDER BY table_name;
"""

with conn.cursor() as cur:
    cur.execute(query)
    tables = cur.fetchall()
    table_list = [row[0] for row in tables]
    print("Public tables in the database:")
    print(table_list)


Public tables in the database:
['boxscoreadvancedv3_playerstats', 'gleague_games', 'gleague_players', 'gleague_teams', 'mastergames', 'masterplayers', 'nba_games', 'nba_players', 'nba_teams', 'test_games_g_league', 'test_games_nba', 'test_games_wnba', 'wnba_games', 'wnba_players', 'wnba_teams']


In [12]:
# Query all games from test_games_nba and display as a DataFrame
query = "SELECT * FROM wnba_games;"

games_df = None
with conn.cursor() as cur:
    cur.execute(query)
    rows = cur.fetchall()
    colnames = [desc[0] for desc in cur.description]
    games_df = pd.DataFrame(rows, columns=colnames)
    print(f"Pulled {len(games_df)} rows from test_games_nba.")
    display(games_df.head())


Pulled 6821 rows from test_games_nba.


,game_id,season_id,game_date,home_team_id,home_team_abbreviation,home_team_name,away_team_id,away_team_abbreviation,away_team_name,matchup,home_wl,away_wl,season_type,league_name,created_at,updated_at
0,1029700110,101997,1997-08-24,1611661320,LAS,Los Angeles Sparks,1611661317,PHO,Phoenix Mercury,PHO vs. LAS,L,W,Regular Season,WNBA,2025-08-11 19:04:44.047278,2025-08-11 19:04:44.047278
1,1029700111,101997,1997-08-24,1611661314,CHA,Charlotte Sting,1611661319,UTA,Utah Starzz,UTA @ CHA,W,L,Regular Season,WNBA,2025-08-11 19:04:44.047278,2025-08-11 19:04:44.047278
2,1029700109,101997,1997-08-24,1611661315,CLE,Cleveland Rockers,1611661313,NYL,New York Liberty,NYL vs. CLE,L,W,Regular Season,WNBA,2025-08-11 19:04:44.047278,2025-08-11 19:04:44.047278
3,1029700112,101997,1997-08-24,1611661318,SAC,Sacramento Monarchs,1611661316,HOU,Houston Comets,HOU vs. SAC,W,L,Regular Season,WNBA,2025-08-11 19:04:44.047278,2025-08-11 19:04:44.047278
4,1029700108,101997,1997-08-23,1611661315,CLE,Cleveland Rockers,1611661313,NYL,New York Liberty,NYL @ CLE,W,L,Regular Season,WNBA,2025-08-11 19:04:44.047278,2025-08-11 19:04:44.047278


In [ ]:
# Fixed ScoreboardV2 call with proper error handling
from datetime import datetime
import time

# Define variables that were missing
current_date = datetime(2025, 8, 11)  # Use a known date with games
league_id = '10'  # WNBA (active season)

print(f"🏀 Attempting to fetch scoreboard...")
print(f"   Date: {current_date.strftime('%m/%d/%Y')}")
print(f"   League: {'WNBA' if league_id == '10' else 'NBA' if league_id == '00' else 'G-League'}")

try:
    # Add a small delay to avoid rate limiting
    time.sleep(1)
    
    scoreboard_data = nbaapi.scoreboardv2.ScoreboardV2(
        game_date=current_date.strftime('%m/%d/%Y'),
        league_id=league_id
    )
    
    # Check if we got data
    games_df = scoreboard_data.get_data_frames()[0]
    
    print(f"✅ Success! Found {len(games_df)} games")
    
    if len(games_df) > 0:
        print(f"📊 Available columns: {list(games_df.columns)}")
        display(games_df.head(3))
    else:
        print("📅 No games found for this date/league combination")
        
except Exception as e:
    print(f"❌ Error details: {str(e)}")
    print(f"❌ Error type: {type(e).__name__}")
    
    # Common solutions for different error types
    if "timeout" in str(e).lower():
        print("💡 Solution: API timeout - try again in a few seconds")
    elif "rate" in str(e).lower() or "429" in str(e):
        print("💡 Solution: Rate limited - wait 60 seconds before trying again")
    elif "connection" in str(e).lower():
        print("💡 Solution: Connection issue - check your internet connection")
    elif "not found" in str(e).lower() or "404" in str(e):
        print("💡 Solution: Try a different date or league_id")
    else:
        print("💡 Solution: Try different parameters or check NBA API status")
# Set parameters
current_date = datetime(2025, 1, 6)  # Example date - adjust as needed
league_id = '00'  # NBA='00', WNBA='10', G-League='20'

try:
    print(f"🏀 Fetching scoreboard for {current_date.strftime('%m/%d/%Y')} (League ID: {league_id})")
    
    scoreboard_data = nbaapi.scoreboardv2.ScoreboardV2(
        game_date=current_date.strftime('%m/%d/%Y'),
        league_id=league_id
    )
    
    # Get the games data
    games_df = scoreboard_data.get_data_frames()[0]
    
    print(f"✅ Found {len(games_df)} games")
    print(f"📊 Columns available: {list(games_df.columns)}")
    
    if len(games_df) > 0:
        display(games_df.head())
    else:
        print("📅 No games scheduled for this date")
        
except Exception as e:
    print(f"❌ Error fetching scoreboard: {e}")
    print("💡 Try adjusting the date or league_id")

🏀 Fetching scoreboard for 01/06/2025 (League ID: 00)
❌ Error fetching scoreboard: 'WinProbability'
💡 Try adjusting the date or league_id


In [27]:
games_df.sort_values(by='game_date')

,game_id,season_id,game_date,home_team_id,home_team_abbreviation,home_team_name,away_team_id,away_team_abbreviation,away_team_name,matchup,home_wl,away_wl,season_type,league_name,created_at,updated_at
111,1029700003,101997,1997-06-21,1611661319,UTA,Utah Starzz,1611661318,SAC,Sacramento Monarchs,SAC @ UTA,L,W,Regular Season,WNBA,2025-08-11 19:04:44.047278,2025-08-11 19:04:44.047278
110,1029700002,101997,1997-06-21,1611661316,HOU,Houston Comets,1611661315,CLE,Cleveland Rockers,CLE vs. HOU,W,L,Regular Season,WNBA,2025-08-11 19:04:44.047278,2025-08-11 19:04:44.047278
109,1029700001,101997,1997-06-21,1611661320,LAS,Los Angeles Sparks,1611661313,NYL,New York Liberty,NYL @ LAS,L,W,Regular Season,WNBA,2025-08-11 19:04:44.047278,2025-08-11 19:04:44.047278
108,1029700004,101997,1997-06-22,1611661317,PHO,Phoenix Mercury,1611661314,CHA,Charlotte Sting,CHA @ PHO,W,L,Regular Season,WNBA,2025-08-11 19:04:44.047278,2025-08-11 19:04:44.047278
107,1029700006,101997,1997-06-23,1611661313,NYL,New York Liberty,1611661318,SAC,Sacramento Monarchs,SAC vs. NYL,W,L,Regular Season,WNBA,2025-08-11 19:04:44.047278,2025-08-11 19:04:44.047278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1546,1022500202,,2025-08-10,1611661317,None,None,1611661330,None,None,,None,None,None,None,2025-08-11 22:50:09.696294,2025-08-11 23:15:47.676736
1402,1022500201,,2025-08-10,1611661320,None,None,1611661328,None,None,,None,None,None,None,2025-08-11 22:50:09.696294,2025-08-11 23:15:47.676736
1257,1022500200,,2025-08-10,1611661321,None,None,1611661322,None,None,,None,None,None,None,2025-08-11 22:50:09.696294,2025-08-11 23:15:47.676736
1208,1022500199,,2025-08-10,1611661313,None,None,1611661324,None,None,,None,None,None,None,2025-08-11 22:50:09.696294,2025-08-11 23:15:47.676736


In [19]:
scoreboard_data = nbaapi.scoreboardv2.ScoreboardV2(
                        game_date=current_date.strftime('%m/%d/%Y'),
                        league_id=league_id
                    )

NameError: name 'current_date' is not defined

In [ ]:
# Troubleshooting ScoreboardV2 - Try multiple approaches
import time

print("🔧 TROUBLESHOOTING SCOREBOARD API")
print("=" * 50)

# Test different combinations to find what works
test_cases = [
    {"date": "08/11/2025", "league": "10", "name": "WNBA Yesterday"},
    {"date": "08/10/2025", "league": "10", "name": "WNBA Day Before"},
    {"date": "08/12/2025", "league": "00", "name": "NBA Today"},
    {"date": "02/15/2025", "league": "00", "name": "NBA Mid-Season"},
]

successful_calls = []

for i, test in enumerate(test_cases):
    print(f"\n🧪 Test {i+1}: {test['name']}")
    print(f"   Date: {test['date']}, League: {test['league']}")
    
    try:
        # Small delay between API calls
        if i > 0:
            time.sleep(2)
            
        scoreboard = nbaapi.scoreboardv2.ScoreboardV2(
            game_date=test['date'],
            league_id=test['league']
        )
        
        games = scoreboard.get_data_frames()[0]
        print(f"   ✅ SUCCESS: {len(games)} games found")
        successful_calls.append(test)
        
        # Show sample data for first successful call
        if len(successful_calls) == 1 and len(games) > 0:
            print(f"   📊 Sample columns: {list(games.columns)[:8]}")
            
    except Exception as e:
        error_msg = str(e)[:100]
        print(f"   ❌ FAILED: {error_msg}...")

print(f"\n📈 SUMMARY:")
print(f"   Successful calls: {len(successful_calls)}")
print(f"   Failed calls: {len(test_cases) - len(successful_calls)}")

if successful_calls:
    print(f"   ✅ Working combinations:")
    for success in successful_calls:
        print(f"      - {success['name']}: {success['date']}, League {success['league']}")
else:
    print(f"   ⚠️ All calls failed - possible API issues or rate limiting")
    print(f"   💡 Try again in a few minutes or check NBA API status")

In [ ]:
# ScoreboardV2 Examples for Different Leagues
print("🏀 NBA Scoreboard Examples")
print("=" * 40)

# Different ways to call ScoreboardV2
examples = [
    {
        'name': 'NBA Today',
        'date': '08/12/2025',  # Today's date
        'league_id': '00',
        'description': 'NBA games for today'
    },
    {
        'name': 'WNBA Recent',  
        'date': '08/11/2025',  # Yesterday - when WNBA had games
        'league_id': '10',
        'description': 'WNBA games from yesterday'
    },
    {
        'name': 'G-League Example',
        'date': '02/15/2025',  # During G-League season
        'league_id': '20', 
        'description': 'G-League games (when in season)'
    }
]

for example in examples:
    print(f"\n📋 {example['name']} - {example['description']}")
    print(f"   Date: {example['date']}, League ID: {example['league_id']}")
    
    try:
        scoreboard = nbaapi.scoreboardv2.ScoreboardV2(
            game_date=example['date'],
            league_id=example['league_id']
        )
        
        games = scoreboard.get_data_frames()[0]
        print(f"   ✅ {len(games)} games found")
        
        if len(games) > 0:
            print(f"   📊 Sample game: {games.iloc[0]['GAMECODE'] if 'GAMECODE' in games.columns else 'N/A'}")
            
    except Exception as e:
        print(f"   ⚠️ Error: {str(e)[:100]}...")
        
print(f"\n💡 League IDs: NBA='00', WNBA='10', G-League='20'")
print(f"📅 Date format: MM/DD/YYYY")

In [ ]:
# Extract Specific Data from ScoreboardV2
# Let's get a working example with WNBA data (since WNBA season is active)

try:
    print("🏀 Getting WNBA scoreboard data...")
    
    # Use a date when we know WNBA had games
    wnba_scoreboard = nbaapi.scoreboardv2.ScoreboardV2(
        game_date='08/11/2025',  # Yesterday - when we collected 11 WNBA games
        league_id='10'  # WNBA
    )
    
    # ScoreboardV2 returns multiple DataFrames
    all_dataframes = wnba_scoreboard.get_data_frames()
    
    print(f"📊 ScoreboardV2 returns {len(all_dataframes)} DataFrames:")
    for i, df in enumerate(all_dataframes):
        print(f"   DataFrame {i}: {len(df)} rows, {len(df.columns)} columns")
        if len(df.columns) > 0:
            print(f"      Columns: {list(df.columns)[:5]}...")  # Show first 5 columns
    
    # The main games data is usually in the first DataFrame
    if len(all_dataframes) > 0 and len(all_dataframes[0]) > 0:
        games_df = all_dataframes[0]
        
        print(f"\n🎯 Main Games Data (DataFrame 0):")
        print(f"   Games found: {len(games_df)}")
        
        # Show key columns if they exist
        key_columns = ['GAME_DATE_EST', 'GAMECODE', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 
                      'GAME_STATUS_TEXT', 'HOME_TEAM_SCORE', 'VISITOR_TEAM_SCORE']
        
        available_columns = [col for col in key_columns if col in games_df.columns]
        
        if available_columns:
            print(f"   Key data preview:")
            display(games_df[available_columns].head())
        else:
            print(f"   All columns: {list(games_df.columns)}")
            display(games_df.head())
    else:
        print("   No games found in main DataFrame")
        
except Exception as e:
    print(f"❌ Error: {e}")
    print("💡 This might happen if the API is rate-limited or the date has no games")